In [1]:
import os
from dotenv import load_dotenv
from pprint import pprint

from stravalib import Client

load_dotenv(override=True)

True

In [5]:
client = Client(
    access_token=os.getenv("STRAVA_ACCESS_TOKEN"),
    token_expires=int(os.getenv("STRAVA_TOKEN_EXPIRES")),
    refresh_token=os.getenv("STRAVA_REFRESH_TOKEN"),
)
athlete = client.get_athlete()

In [ ]:
# https://stravalib.readthedocs.io/en/latest/reference/api/stravalib.model.DetailedAthlete.html#stravalib.model.DetailedAthlete
print(athlete.id)
print(athlete.firstname)
print(athlete.lastname)
print(athlete.sex)
print(athlete.athlete_type)
print(athlete.follower_count)
print(athlete.bikes)
print(athlete.shoes)

In [ ]:
# athlete.model_dump()

In [ ]:
# https://stravalib.readthedocs.io/en/latest/reference/api/stravalib.model.AthleteStats.html#stravalib.model.AthleteStats
print(athlete.stats.all_run_totals)
print(athlete.stats.ytd_run_totals)
print(athlete.stats.recent_run_totals)
print(athlete.stats.biggest_ride_distance)

In [ ]:
athlete.stats.model_dump()

In [6]:
activities = client.get_activities(limit=15)

In [7]:
for activity in activities:
    print(f"{activity.name} | {activity.kudos_count} | {activity.distance} | {activity.average_heartrate}")

2 x Coste Belle 🐏 | 4 | 12049.0 | 163.8
Fractionné #58 📫 | 12 | 11104.3 | 161.1
Tourisme dans le Gard #3 : Uzès 🏰 | 12 | 12826.6 | 161.9
Les Gorges de l'Ardèche 🐐 | 14 | 143002.0 | 138.1
1h30 de Pluie 🫠 | 14 | 15931.8 | 163.4
Montée Sympa 😁 | 11 | 12343.8 | 156.8
Tourisme dans le Gard #2 : Dions 🧗🏾 | 10 | 51179.4 | 129.5
Fractionné #57 🐌 | 13 | 9975.8 | 153.9
Pas de pluie 🙂 | 12 | 12118.3 | 159.5
Promenade de Récupération 🙂 | 12 | 20358.8 | 121.1
Rando des Côtes du Rhône Primeurs 🍷 | 11 | 34125.8 | 125.9
Boucle par La Capelle 📃 | 10 | 27404.9 | 135.3
Fractionné #56 🚜 | 13 | 10099.1 | 158.9
Tourisme dans le Gard #1 : Lussan 🌬️ | 14 | 48472.3 | 134.6
De Mieux en Mieux 😄 | 15 | 10134.3 | 156.6


In [ ]:
for activity in activities:
    print(activity.model_dump())
    break

In [8]:
import polars as pl

data = []
for activity in activities:
    data.append(activity.model_dump())

df = pl.DataFrame(data)
df

id,achievement_count,athlete,athlete_count,average_speed,average_watts,comment_count,commute,device_watts,distance,elapsed_time,elev_high,elev_low,end_latlng,external_id,flagged,gear_id,has_kudoed,hide_from_home,kilojoules,kudos_count,manual,map,max_speed,max_watts,moving_time,name,photo_count,private,sport_type,start_date,start_date_local,start_latlng,timezone,total_elevation_gain,total_photo_count,trainer,type,upload_id,upload_id_str,weighted_average_watts,workout_type,utc_offset,location_city,location_state,location_country,pr_count,suffer_score,has_heartrate,average_heartrate,max_heartrate,average_cadence,from_accepted_tag,visibility
i64,i64,struct[2],i64,f64,null,i64,bool,null,f64,i64,f64,f64,list[f64],str,bool,str,bool,null,null,i64,bool,struct[3],f64,null,i64,str,i64,bool,str,"datetime[μs, UTC]","datetime[μs, UTC]",list[f64],str,f64,i64,bool,str,i64,str,null,null,f64,null,null,null,i64,null,bool,f64,i64,f64,bool,str
16711236450,0,"{88953196,1}",1,3.143,null,0,false,null,12049.0,3833,175.3,110.8,"[44.043407, 4.453185]","""garmin_ping_511440095580""",false,"""g24116397""",false,null,null,4,false,"{""a16711236450"",null,""sfykG_xdZBB@ROJe@@w@UgAC]NWTU|@Qd@WjAO^u@vA]HIVBr@FtD@tAEt@EHKBc@De@A_@Sw@eAe@{@QaBWW[u@OS_@Ig@?c@Dc@LGCCKL{GP}@^eEBy@DWt@Ql@C`@DEhA@b@OfAGz@IXQ`CGd@QTCT?xBGVGJ@~AKXAPA|@FbAEV@P`@pARu@LAbBPNRCrA?zBIlBC|@Ed@@PEhAM`@y@`@}AL{@PYY[MkAGkB]GBI`@KFy@]eBa@_@WAGFwBIu@WgAIs@[cIG{Fg@eKEwBGGkA?MWY_CEIQCgDVa@MKc@u@oE[[kAu@aAe@gB^i@FICECEYGaAS_@c@QwASk@@wAe@i@?}BR_Bf@sA@CJAlHOhE]rDIZE@e@EsA]Mg@Ow@Og@Io@?mAFwBGo@WgAOW_CTg@IwBu@yAP_@Ak@K@^KTHn@W`BGfBHl@NRRxB\hC?DMNSFAFAp@Ip@D`@Nl@PfAb@fBDlA`@jCd@lAnAzBp@pB\vANZZd@p@hB^r@d@`BNx@dBjE`@~AR^TPHRDn@zAxEN~@d@zB|@lBj@fBx@xDb@bBFz@Lr@p@tB~@nDb@`AjAhB^XRF?BK?{@}@u@iAg@eAwAqFUo@Oy@Gs@i@oBaA{Ey@sBo@sBWoAGq@Mg@c@uAs@gBQaAHRHt@jAdDV|@NlAf@xBvApDjAnFRt@\~BjBnGbAvBr@dANN\H@@?FGAc@a@c@q@}@}A]y@}AsFe@mCUw@mAyFiByEQ}@UiBMi@k@}Ac@aAMm@Gq@NSd@QlBQ|Ae@fD]jA@dCP`@THHZLtEMnBJh@L`EJ`Ef@PDLP\x@RVHAFI?W?iBBcHDqAHc@BCZDtBf@z@^HHZj@Jn@PtARr@x@~AXlARh@BDTJn@n@B??EGU@_@HM`@H^Cb@MNKx@Pn@DNO?oBIw@@m@Fo@OaAAi@WUWEOXCVWt@a@fBu@NWLCAM]JWr@c@\ODFAf@M^[?g@XECEKKm@OOY}C?yAGs@Hs@F_ABcBFWZaCAiAJw@EW@ERKDDn@xAT{@BA~@F~@?z@Ij@Bl@IXODkCE_BAmCHING\[^w@Vo@ZuBLa@^]XKfABlAPh@KBE""}",4.133,null,3833,"""2 x Coste Belle 🐏""",0,false,"""TrailRun""",2025-12-11 07:45:31 UTC,2025-12-11 08:45:31 UTC,"[44.043465, 4.453289]","""(GMT+01:00) Europe/Paris""",240.7,0,false,"""Run""",17801135000,"""17801135000""",null,null,3600.0,null,null,null,0,null,true,163.8,189,84.8,false,"""everyone"""
16701720797,0,"{88953196,1}",1,3.486,null,0,false,null,11104.3,3370,140.7,98.2,"[44.043385, 4.453184]","""garmin_ping_511094659941""",false,"""g19517047""",false,null,null,12,false,"{""a16701720797"",null,""_fykGowdZXh@?DCn@?nBElBk@|LBjGCnD@z@NtCLPD?HCdAuAb@c@hBwBnAwC\u@x@gA`ByAVa@Tk@nAqCp@eB`A{BRo@b@kAPk@Kc@kBeCwCgEiB}CiDkFw@{Ak@o@SOsCuAi@MgBq@cB[{Bo@{By@uAq@i@Q_BY_BGeCYk@QIG]_@Yi@Y_@yBkC{C_EMMsCaBa@[y@s@M_@?JRVf@^hDtB\\rA~Ap@`ARh@x@lAP\EFOC?Eu@@q@Gm@K}@Uo@[SE}@W{@WWCSG_BQK?qDe@g@CZBHAHBAAo@E{AO}@Eq@EiASoCq@{Cc@}Dw@oB[kAGeDNcCRe@@oBGuABi@H}Ah@kCh@mI~@kBNu@CsB?c@AwFi@eAQMDC\P|BFvALjAXjBNp@Nb@`@x@Tp@lAxEx@rCn@jC`@fAX`Al@fBPp@`@|Bf@pBl@~Ax@fCVzAr@nFf@vCnAfGNd@Vh@l@dAzAjBvBrCbAfB^x@nDbFbAlB`A`Bz@jBr@fA|ArBXf@~AdDdC|ETh@h@bBd@fA~@fB~@lBzBpDTd@Pj@LR`@RBDVxAJb@\rAVd@`@T`Ar@~A`AjDfBdATb@RrAVv@Jb@Lr@^xB`AdBh@NJh@b@HAFMb@eBh@gBj@qDPUb@Yt@ApBBn@?lCNjA@d@?HI^sGFcBDqBB{F@OVSrBm@|@Q|@GpBAf@@z@Et@@^Q?WIcAE}A@s@PI?KMk@?eAEq@S]Kk@KIe@Um@Ke@CMFIt@C?K]Aq@GWc@Sm@ISO[Ea@]YDCC@e@Ls@Fy@R{@JS@q@JiAFc@Pu@|@cIJ]Za@n@e@R]DS@WGuAA_A@o@?_ABy@CmHBeA\iHHsCCuEAOKQ""}",5.45,null,3185,"""Fractionné #58 📫""",0,false,"""Run""",2025-12-10 07:45:22 UTC,2025-12-10 08:45:22 UTC,"[44.04336, 4.453207]","""(GMT+01:00) Europe/Paris""",57.3,0,false,"""Run""",17791526251,"""17791526251""",null,null,3600.0,null,null,null,0,null,true,161.1,192,85.6,false,"""everyone"""
16694771995,0,"{88953196,1}",1,3.212,null,0,false,null,12826.6,4017,142.9,83.8,"[44.043383, 4.453201]","""garmin_ping_51083

In [20]:
df_test = (
    df.select(["sport_type", "distance", "start_date_local"])
    .sort("start_date_local")
    .with_columns(pl.col("start_date_local").dt.week().alias("week"))
    .drop("start_date_local")
    .group_by(["week", "sport_type"]).agg(pl.col("distance").sum()).sort("week")
)
df_test

week,sport_type,distance
i8,str,f64
47,"""MountainBikeRide""",34125.8
47,"""Run""",20233.4
47,"""Ride""",96236.0
48,"""Run""",22319.6
48,"""Ride""",51179.4
48,"""TrailRun""",12118.3
49,"""Ride""",143002.0
49,"""TrailRun""",15931.8
50,"""TrailRun""",12049.0


In [ ]:
import plotly.express as px
run_df = df.filter(pl.col("type") == "Run")
fig = px.line(df, x="start_date_local", y="distance")
fig.show()

In [ ]:
df.to_dicts()

In [ ]:
# client.get_athlete_stats()
# client.get_athlete_zones()

In [ ]:
for kom in client.get_athlete_koms(athlete.id, limit=3):
    pprint(kom.model_dump())